In [1]:
import pandas as pd
from path import Path

In [3]:
track_cmc_merged_df = pd.read_csv(Path(f"../../../../data/cleandata/track_cmc_merged_df.csv"))

In [19]:
track_cmc_merged_df.iloc[704]

rank_cnc                               1380
Name                                 UChain
Ticker                                  UCN
market_cap                          155,884
price                              0.000511
volume24                             50,389
pct1h                                     1
pct24h                                -0.66
pct7d                                 -0.76
country                           Singapore
platform                           Ethereum
pre_ico_end             2017-09-28 03:00:00
pre_ico_start           2017-09-28 03:00:00
rating                                    5
status                               Closed
Name_y                          VerifyUnion
type                                    ICO
End                     2017-12-03 00:00:00
Start                   2017-10-03 00:00:00
Duration         61 days 00:00:00.000000000
pre_Duration      0 days 00:00:00.000000000
Name: 704, dtype: object

In [49]:
ticker_sentiments_df = pd.read_csv(Path(f"../../../../data/cleandata/ticker_sentiments.csv"))

In [50]:
ticker_sentiments_df.rename(columns = {'ticker':'Ticker'}, inplace = True)

In [52]:
merged_inner = pd.merge(left=track_cmc_merged_df,right=ticker_sentiments_df, left_on='Ticker', right_on='Ticker')
merged_inner.head()

,rank_cnc,Name,Ticker,market_cap,price,volume24,pct1h,pct24h,pct7d,country,...,type,End,Start,Duration,pre_Duration,compound,negative,neutral,positive,no_of_posts
0,7,Binance Coin,BNB,"3,277,372,658",21.07,"231,846,713",-0.46,-1.33,2.49,Japan,...,ICO,2017-07-21 07:00:00,2017-07-01 07:00:00,20 days 00:00:00.000000000,0 days 00:00:00.000000000,0.563773,0.096670,0.041410,0.861890,100
1,8,EOS,EOS,"3,214,893,394",3.42,"1,735,988,258",0.11,-1.49,-2.55,Cayman Islands,...,ICO,2018-06-12 00:00:00,2017-06-26 16:00:00,350 days 08:00:00.000000000,0 days 00:00:00.000000000,0.828470,0.032200,0.858550,0.109270,100
2,13,Cardano,ADA,"1,105,888,653",0.042654,"53,437,224",-0.50,-1.72,-2.12,Switzerland,...,ICO,2017-01-01 00:00:00,2015-09-01 00:00:00,488 days 00:00:00.000000000,0 days 00:00:00.000000000,0.589474,0.098387,0.032946,0.868753,93
3,18,Tezos,XTZ,"821,746,518",1.24,"44,390,717",-0.25,10.13,2.47,United States of America,...,ICO,2017-07-14 12:00:00,2017-07-01 12:00:00,13 days 00:00:00.000000000,0 days 00:00:00.000000000,0.663939,0.021939,0.878857,0.099327,49
4,19,Cosmos,ATOM,"760,265,147",3.99,"147,439,544",-1.62,1.29,2.14,Canada,...,ICO,2017-04-06 13:00:00,2017-04-06 12:00:00,0 days 01:00:00.000000000,0 days 00:00:00.000000000,0.340496,0.060687,0.023224,0.916194,67


In [53]:
merged_inner.shape

(364, 26)

In [58]:
def custom_index_success(df):
    for index, row in df.iterrows():
#         longevity = ''' want to subtract ico_end from datetimenow'''
#         current_price = df['']

In [59]:
custom_index_success(merged_inner)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [55]:
def check_what_matches(df1,df2):
    duplicates = []
    
    for track in df1.index.unique().tolist():
        for watch in df2.index.unique().tolist():
            if track == watch:
                duplicates.append(track)
            else:
                continue
    return duplicates
#len(check_what_matches(track_cmc_merged_df,ticker_sentiments_df))

In [14]:
file_path = Path("../../../../data/cleandata/sentiment_cmc_track_merge.csv")
merged_inner.to_csv(file_path, index=False)